# collect data from ppt beauty

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import json
import random
import re
from concurrent.futures import ThreadPoolExecutor
import threading
import sys
payload = {
    'from':'/bbs/Beauty/index.html',
    'yes':'yes'
}
rs = requests.session()
res = rs.post("https://www.ptt.cc/ask/over18",data = payload)

In [5]:
def get_first_post_index():
    i=1
    while True:
        content = rs.get(f"https://www.ptt.cc/bbs/Beauty/index{i}.html")
        soup = BeautifulSoup(content.text, 'html.parser')
        articles = soup.select('div.r-ent')
        for article in articles:
            title = article.find("div", class_="title")
            if title.a is None:
                continue
            url = str(title.a['href'])
            if url == "/bbs/Beauty/M.1640974182.A.7DB.html":
                return i
        i+=1
def get_last_post_index(start_index):
    index = start_index
    tmp_date =""
    while True:
        content = rs.get(f"https://www.ptt.cc/bbs/Beauty/index{index}.html")
        soup = BeautifulSoup(content.text, 'html.parser')
        articles = soup.select('div.r-ent')
        for article in articles:
            date = article.select_one("div.date").text.strip()
            if (tmp_date =="")&(date=="1/01"):
                tmp_date = date
            if (date=="1/01") & (tmp_date =="12/31"):
                return index
            if tmp_date !="":
                tmp_date = date
        index += 1
start_index = get_first_post_index()
end_index = get_last_post_index(start_index)

In [41]:
start_index,end_index

(3642, 3951)

# Method 1(score 0~100) :
* Get image link from ptt
* 以推文數作為image score

In [4]:

image_score = {i : [] for i in range(0,101)}
lock = threading.Lock()
def get_image(index):
    global image_score, lock
    content = rs.get(f"https://www.ptt.cc/bbs/Beauty/index{index}.html")
    soup = BeautifulSoup(content.text, 'html.parser')
    articles = soup.select('div.r-ent')
    for article in articles:
        article_tag = article.select_one(".nrec").text
        if article_tag=="":
            continue
        url = "https://www.ptt.cc/" +article.select_one('.title').a['href']
        content = rs.get(url)
        soup = BeautifulSoup(content.text, 'html.parser')
        main = soup.select_one("div#main-content")
        excluded_tag = main.select(".push,.f2,div.article-mataline,.article-mataline-right")
        for tag in excluded_tag:
            tag.decompose()
        all_link = main.select("a")
        for link in all_link:
            if re.match(r'.*\.(jpg|png|jpeg)$',link['href']):
                with lock:
                    if article_tag=="爆":
                        image_score[100].append(link['href'])
                    elif not article_tag.isnumeric():
                        image_score[0].append(link['href'])
                    else:
                        image_score[int(article_tag)].append(link['href'])
        delay_choices = [0.2,0.3,0.4,0.5] 
        delay = random.choice(delay_choices) 
        time.sleep(delay)
with ThreadPoolExecutor(max_workers=10) as executor:
    for i in range(2900,4002):
        executor.submit(get_image,i)
output1 = "image_score.json"
# output2 = "not_popular_image.json"
with open(output1,'w') as f:
    json.dump(image_score,f)
# with open(output2,'w') as f:
#     json.dump({
#         "image_urls":not_popular_image
#     },f)

* Convert link to image file
* check if image can be read by Image.open()

In [ ]:
import json
import urllib.request
from pathlib import Path
from PIL import Image
import os
with open("./image_score.json") as f:
    image_score = json.load(f)
for i in range(1,101):
    data = image_score[i]
    folder_path = f"data/{i}/"
    Path(folder_path).mkdir(parents=True, exist_ok=True)
    for i in range(len(data)):
        try:
            url = data[i]
            filename = folder_path/data[i].split("/")[-1]
            urllib.request.urlretrieve(url, filename)
        except Exception as err:
            print(f"Sth. wrong with {url}, error : {err}")
for i in range(1,101):
    image_list = list(Path("./data/{i}").glob("*"))
    for image in image_list:
        try:
            img = Image.open(image)
        except Exception as err:
            print(err)
            os.remove(image)

* split data into train & test data(from data folder to new_data folder)

In [ ]:
import shutil
for i in range(0,101):
    image_list = list(Path("./data/{i}").glob("*"))
    for k,image in enumerate(image_list):
        if k>len(image_list)/5:
            shutil.copy(image,"./new_data/train/{i}/")
        else:
            shutil.mocopyve(image,"./new_data/test/{i}/")

* crop the face from image

In [ ]:
import face_recognition
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
import threading
import os 

for i in range(0,101):
    train_image_list = list(Path(f"new_data/train/{i}/").glob("*"))
    test_image_list = list(Path(f"new_data/test/{i}/").glob("*"))
    for l in [train_image_list,test_image_list]:
        for image in l:
            # print(image)
            img = face_recognition.load_image_file(image)
            face_locations = face_recognition.face_locations(img)
            if len(face_locations) != 1 :
                # print("not a good image")
                os.remove(image)
                continue
            else:
                top, right, bottom, left = face_locations[0]
                face_img = img[top:bottom, left:right]
                pil_image = Image.fromarray(face_img)
                pil_image.save(image)

# Method 2(popular & not popular) :
* modify method 1's code to scrape link from ptt (get popular_image.json & not_popular_image.json)
* Split data into train & test
* convert link to image file


In [32]:
import json
import urllib.request
from pathlib import Path
from urllib.error import HTTPError
with open("./popular_image.json") as f:
    popular_image = json.load(f)
with open("./not_popular_image.json") as f:
    not_popular_image = json.load(f)
not_popular_image = not_popular_image['image_urls'][:15000]
popular_image = popular_image['image_urls'][:15000]

for i in range(len(popular_image)):
    if i<(len(popular_image)*4/5):
        try:
            url = popular_image[i]
            filename = "train/popular/"+popular_image[i].split("/")[-1]
            urllib.request.urlretrieve(url, filename)
        except Exception as err:
            print(f"Sth. wrong with {url}, error : {err}")
        # continue
    else:
        try:
            url = popular_image[i]
            filename = "test/popular/"+popular_image[i].split("/")[-1]
            urllib.request.urlretrieve(url, filename)
        except Exception as err:
            print(f"Sth. wrong with {url}, error : {err}")
for i in range(len(not_popular_image)):
    if i<(len(not_popular_image)*4/5):
        try:
            url = not_popular_image[i]
            filename = "train/not_popular/" + not_popular_image[i].split("/")[-1]
            urllib.request.urlretrieve(url, filename)
        except Exception as err:
            print(f"Sth. wrong with {url}, error : {err}")
            
    else:
        try:
            url = not_popular_image[i]
            filename = "test/not_popular/"+not_popular_image[i].split("/")[-1]
            urllib.request.urlretrieve(url, filename)
        except Exception as err:
            print(f"Sth. wrong with {url}, error : {err}")

Sth. wrong with https://imgur.dcard.tw/IIwP9Y4h.png, error : HTTP Error 403: Forbidden
Sth. wrong with https://imgur.com/undefined.jpg, error : HTTP Error 404: Not Found
Sth. wrong with https://i2.kknews.cc/SIG=1mjs6kc/1ns40000q02nopp1n5s5.jpg, error : HTTP Error 403: Forbidden
Sth. wrong with https://i1.kknews.cc/SIG=318r8na/19sr00069o3rq6so2706.jpg, error : HTTP Error 403: Forbidden
Sth. wrong with https://img.bingfeng.tw/bingfeng/forum/201811/19/210508q7x70trrmvsmotjr.jpg, error : HTTP Error 403: Forbidden
Sth. wrong with https://assets.juksy.com/files/articles/91916/800x_100_w-5d11dea5c11ba.png, error : HTTP Error 403: Forbidden
Sth. wrong with https://file.jichengguandao.com/uploads/2020/07/07/nzuxf25ot0m.png, error : <urlopen error [WinError 10054] 遠端主機已強制關閉一個現存的連線。>
Sth. wrong with https://img.chinatimes.com/newsphoto/2019-05-08/656/20190508001595.jpg, error : <urlopen error [Errno 11001] getaddrinfo failed>
Sth. wrong with https://img.chinatimes.com/newsphoto/2019-05-08/656/201

* get number of data

In [35]:
import os
import glob
mode = ["train", "test"]
t= ["popular", "not_popular"]
for m in mode:
    for i in t:
        folder = f"./{m}/{i}/"
        data = glob.glob(os.path.join(folder,'*'))
        print(f"{m} {i}:",len(data))

train popular: 11807
train not_popular: 11522
test popular: 2985
test not_popular: 2980
